<a href="https://colab.research.google.com/github/hiba6873/Compiler-C--/blob/main/RealTime_Retail_Analytics_and_Product_Recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Real-Time Retail Analytics and Product Recommender
Team members:
* Nour El Moujahid
* Hiba Jebbar
* Ilyass Louzali
* Aya Boumelha

##Milestone 1 (Nour):

Neural Network Workflow (PyTorch): Train deep learning models to predict
purchasing behavior and customer retention based on session logs and historical purchase
data.


###Part 1: Load Kaggle dataset into a Pandas dataframe
Kaggle dataset site: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset

In [1]:
# Opendatasets is a Python library for downloading datasets from online sources like Kaggle
# Install the library
!pip install opendatasets --upgrade

In [2]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
import opendatasets as od
import pandas as pd

In [3]:
# Download the dataset
# API TOKEN = {"username":"nourelmoujahid","key":"611189b7ac2a80012221c170415923b6"}
data_url = "https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset"
dataset = od.download(data_url)

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: nourelmoujahid
Your Kaggle Key: ··········
Dataset URL: https://www.kaggle.com/datasets/lakshmi25npathi/online-retail-dataset


In [4]:
# Convert the downloaded dataset into a dataframe
file = 'online-retail-dataset/online_retail_II.xlsx'
df = pd.read_excel(file)

In [5]:
# Pandas dataframe loaded successfully
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


###Part 2: Data Analysis

Features' Information:

* `InvoiceNo`: Invoice number. If this code starts with the letter 'c', it indicates a cancellation.

* `StockCode`: Product (item) code. A 5-digit integral number uniquely assigned to each distinct product.

* `Description`: Product (item) name.

* `Quantity`: The quantities of each product (item) per transaction.

* `InvoiceDate`: The day and time when a transaction was generated.

* `UnitPrice`: Product price per unit in sterling (Â£).

* `CustomerID`: Customer number. A 5-digit integral number uniquely assigned to each customer.

* `Country`: The name of the country where a customer resides.

Assumptions about the Features according Nour Ackerman

* `Invoice`: check number of 'c' per country or per `CustomerID`
* `StockCode`: meh, does not seem relevant. better check `Description`
* `Quantity`: important, how much you like the product per `Country` or `CustomerID`
* `InvoiceDate`: no clue my dude, I need to check
* `UnitPrice`: yea why not, how expensiveness affects purchases

In [ ]:
#features' data types
df.dtypes

,0
Invoice,object
StockCode,object
Description,object
Quantity,int64
InvoiceDate,datetime64[ns]
Price,float64
Customer ID,float64
Country,object


In [ ]:
# 525461 rows
# 8 columns
df.shape

(525461, 8)

In [ ]:
df.at[100434, "Country"]

'France'

In [ ]:
# Available countries
df["Country"].unique()

array(['United Kingdom', 'France', 'USA', 'Belgium', 'Australia', 'EIRE',
       'Germany', 'Portugal', 'Japan', 'Denmark', 'Nigeria',
       'Netherlands', 'Poland', 'Spain', 'Channel Islands', 'Italy',
       'Cyprus', 'Greece', 'Norway', 'Austria', 'Sweden',
       'United Arab Emirates', 'Finland', 'Switzerland', 'Unspecified',
       'Malta', 'Bahrain', 'RSA', 'Bermuda', 'Hong Kong', 'Singapore',
       'Thailand', 'Israel', 'Lithuania', 'West Indies', 'Lebanon',
       'Korea', 'Brazil', 'Canada', 'Iceland'], dtype=object)

In [ ]:
len(df["Description"].unique())

4682

In [ ]:
len(df["StockCode"].unique())

4632

In [ ]:
#Total quantity
df.groupby(["Country", "Description"])["Quantity"].sum().reset_index(name="TotalQuantity").sort_values(by="TotalQuantity", ascending=False)

,Country,Description,TotalQuantity
19033,United Kingdom,WHITE HANGING HEART T-LIGHT HOLDER,52252
19138,United Kingdom,WORLD WAR 2 GLIDERS ASSTD DESIGNS,51434
15369,United Kingdom,BROCADE RING PURSE,47495
14931,United Kingdom,ASSORTED COLOUR BIRD ORNAMENT,41855
17381,United Kingdom,PACK OF 72 RETRO SPOT CAKE CASES,36393
...,...,...,...
19218,United Kingdom,Zebra invcing error,-9000
19253,United Kingdom,ebay sales,-13630
19270,United Kingdom,missing,-15149
19260,United Kingdom,given away,-20000


##Milestone 2 (Hiba):
Similarity Search: Use high-dimensional similarity search to identify
customers with comparable behavior, enabling collaborative trends and tailored
suggestions.

# **Data Preparation**





In [7]:
# Inspect the data structure first
print(f"Dataset shape: {df.shape}")

Dataset shape: (525461, 8)


In [8]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']


In [9]:
print("\nMissing values per column:")
print(df.isnull().sum())


Missing values per column:
Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64


In [10]:
# Convert InvoiceDate to datetime
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])

In [11]:
# Extract date components for time pattern analysis
df['Year'] = df['InvoiceDate'].dt.year
df['Month'] = df['InvoiceDate'].dt.month
df['Day'] = df['InvoiceDate'].dt.day
df['DayOfWeek'] = df['InvoiceDate'].dt.dayofweek  # 0=Monday, 6=Sunday
df['Hour'] = df['InvoiceDate'].dt.hour

In [28]:
# Filter out returns (negative quantities) and missing customer IDs
df_clean = df[(df['Quantity'] > 0) & (~df['Customer ID'].isna())]

In [34]:
print(df_clean[['Quantity', 'Price']].head())
print(df_clean[['Quantity', 'Price']].dtypes)


   Quantity  Price
0        12   6.95
1        12   6.75
2        12   6.75
3        48   2.10
4        24   1.25
Quantity      int64
Price       float64
dtype: object


In [29]:
df_clean = df[(df['Quantity'] > 0) & (~df['Customer ID'].isna())].copy()
df_clean['TotalAmount'] = df_clean['Quantity'] * df_clean['Price']


In [30]:
# Set the latest date in the dataset for recency calculation
latest_date = df_clean['InvoiceDate'].max()

In [31]:
print(f"\nData ranges from {df_clean['InvoiceDate'].min()} to {latest_date}")
print(f"\nAfter cleaning: {df_clean.shape} (rows, columns)")


Data ranges from 2009-12-01 07:45:00 to 2010-12-09 20:01:00

After cleaning: (407695, 14) (rows, columns)


In [32]:
# Show the cleaned data
df_clean.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Year,Month,Day,DayOfWeek,Hour,TotalAmount
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,2009,12,1,1,7,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,2009,12,1,1,7,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,2009,12,1,1,7,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,2009,12,1,1,7,30.0




*   Feature Set 1: RFM (Recency, Frequency, Monetary) Metrics




Calculate RFM (Recency, Frequency, Monetary) metrics

RFM helps identify a customer's value based on:
- Recency: How recently they purchased (lower days = more engaged)
- Frequency: How often they purchase (higher = more loyal)bold text
- Monetary: How much they spend (higher = more valuable)


In [36]:
print("\nColumn names:")
print(df.columns.tolist())


Column names:
['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour']


In [37]:
print(list(df_clean.columns))


['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country', 'Year', 'Month', 'Day', 'DayOfWeek', 'Hour', 'TotalAmount']


In [48]:
def calculate_rfm_metrics(df, latest_date):
    """
    Calculates RFM metrics for each customer.

    Purpose:
    - Recency identifies which customers are currently engaged
    - Frequency identifies loyal customers
    - Monetary identifies high-value customers
    - Average order value shows spending habits per transaction
    """
    # Group by customer on the cleaned dataset
    customer_group = df.groupby('Customer ID')

    # Recency = Days since last purchase
    recency = (latest_date - customer_group['InvoiceDate'].max()).dt.days.reset_index()
    recency.columns = ['Customer ID', 'Recency']

    # Frequency = Number of unique purchases
    frequency = customer_group['Invoice'].nunique().reset_index()
    frequency.columns = ['Customer ID', 'Frequency']

    # Monetary = Total spent
    monetary = customer_group['TotalAmount'].sum().reset_index()
    monetary.columns = ['Customer ID', 'Monetary']

    # Average order value
    avg_order = customer_group['TotalAmount'].mean().reset_index()
    avg_order.columns = ['Customer ID', 'AvgOrderValue']

    # Merge all metrics
    rfm = recency.merge(frequency, on='Customer ID')\
                 .merge(monetary, on='Customer ID')\
                 .merge(avg_order, on='Customer ID')

    return rfm
# Calculate RFM metrics
rfm_features = calculate_rfm_metrics(df_clean, latest_date)

# Display the RFM metrics for a few customers
print(f"RFM metrics calculated for {len(rfm_features)} customers")
rfm_features.head()

RFM metrics calculated for 4314 customers


,Customer ID,Recency,Frequency,Monetary,AvgOrderValue
0,12346.0,164,11,372.86,11.298788
1,12347.0,2,2,1323.32,18.638310
2,12348.0,73,1,222.16,11.108000
3,12349.0,42,3,2671.14,26.187647
4,12351.0,10,1,300.93,14.330000


Feature Set 2: Time Patterns (Day/Hour of Purchases)

## Milestone 4 (Aya)
Large-Scale Hypothesis Testing: Conduct hypothesis testing across patient
subgroups to uncover statistically significant health factors. Apply corrections for multiple
testing (e.g., Benjamini-Hochberg).

##Milestone 5 (Louzali)
Recommendation Systems: Develop a recommendation engine to propose
lifestyle changes and follow-up actions based on predicted risks and matched patient
cohorts.